In [ ]:
!pip install -e .

In [ ]:
!pip install rich

In [ ]:
import pystac_client
from stac_insitu.filter import filter_trajectories
import shapely

In [ ]:
client = pystac_client.Client.open("https://keewis-stac-insitu.hf.space")
client

In [ ]:
import folium
import folium.plugins
from stac_insitu.visualization import visualize_search

In [ ]:
categories = {
    "bo": "#008888",
    "db": "#880000",
    "ml": "#880088",
    "mo": "#888800",
    "pf": "#000088",
    "rf": "#008800",
    "tg": "#ff88ff",
    "ts": "#81d4fa",
}


def style_function(_, category):
    fill_color = categories.get(category, "#000000")
    style = {
        # "fillColor": fill_color,
        "color": fill_color,
    }

    return {k: v for k, v in style.items() if v is not None}

In [ ]:
geometry = shapely.box(-25, 43, 2, 60)
timespan = ["2023-04-15T00:00:00", "2023-05-02T00:00:00"]

unfiltered = client.search(
    collections=["TS"],
    intersects=geometry,
    datetime=timespan,
    limit=100,
).item_collection()

filtered = filter_trajectories(unfiltered, geometry, timespan)
len(unfiltered), len(filtered)

In [ ]:
from rich.console import Console

console = Console()

In [ ]:
console.print(filtered[0].to_dict())

In [ ]:
m = folium.plugins.DualMap(layout="horizontal", tiles="cartodbpositron")
visualize_search(geometry, unfiltered, m=m.m1)
visualize_search(geometry, filtered, m=m.m2)

m

In [ ]:
import xarray as xr
import fsspec

In [ ]:
fs = fsspec.filesystem("http")
dss = [
    xr.open_dataset(fs.open(item.assets["https"].href), engine="h5netcdf").load()
    for item in filtered
]
dss